In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('./dataset/database_konten.csv')
df

,ID,Judul Artikel,Summary Makanan,Gambar Artikel,Body Artikel,Tags,Link
0,0,Tips Menjaga Kebugaran Lansia yang Perlu Dike...,"“Seiring dengan bertambahnya umur, menjaga ke...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...","[""kesehatan"",""lansia"",""kebugaran""]",https://www.halodoc.com/artikel/tips-menjaga-k...
1,1,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri...",“Ada berbagai manfaat seledri untuk kesehatan...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seledri merupakan sayuran y...","[""kesehatan"",""makanan""]",https://www.halodoc.com/artikel/jarang-diketah...
2,2,9 Obat Herbal yang Ampuh Atasi Sakit Lutut da...,“Obat herbal dapat menjadi pilihan alternatif...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Sakit lutut dan sendi adala...","[""herbal"",""kesehatan"",""obat""]",https://www.halodoc.com/artikel/9-obat-herbal-...
3,3,5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula,“Obat herbal dapat menjadi pilihan alternatif...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,“Yoga adalah olahraga yang terbilang cukup rin...,"[""olahraga"",""yoga"",""pemula""]",https://www.halodoc.com/artikel/5-gerakan-yoga...
4,4,Lakukan 5 Gerakan Yoga Ini untuk Meredakan St...,“Obat herbal dapat menjadi pilihan alternatif...,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Yoga dapat membantu mengura...","[""yoga"",""olahraga"",""kesehatan mental"",""stres""]",https://www.halodoc.com/artikel/lakukan-5-gera...
...,...,...,...,...,...,...,...
185,185,5 Gangguan Kepribadian dengan Rasa Cemas Berl...,“Kebanyakan bayi rewel di malam hari bukan di...,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Gangguan kepribadian adalah...","[""cemas"",""psikologi""]",https://www.halodoc.com/artikel/5-gangguan-kep...
186,186,15 Gejala yang Timbul dari Gangguan Kecemasan,“Kebanyakan bayi rewel di malam hari bukan di...,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Siapa yang tidak mengenal g...","[""cemas"",""psikologi"",""kesehatan mental""]",https://www.halodoc.com/artikel/15-gejala-yang...
187,187,Latihan Pernapasan yang Bisa Meredakan Kecema...,“Kebanyakan bayi rewel di malam hari bukan di...,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Meskipun merupakan perasaan...","[""cemas"",""olahraga""]",https://www.halodoc.com/artikel/latihan-pernap...
188,188,"Makanan Plant Based Bisa Meredakan Kecemasan,...",“Kebanyakan bayi rewel di malam hari bukan di...,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Kesehatan mental sama penti...","[""cemas"",""makanan""]",https://www.halodoc.com/artikel/makanan-plant-...
